<a href="https://colab.research.google.com/github/SiracencoSerghei/DataScienceHW/blob/main/example_kaggle/les_12/Module_12_2_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Питання з таблиці
## Data Science project guides

- Data Camp: https://www.datacamp.com/blog/data-science-use-cases-guide
- 10 Steps: https://towardsdatascience.com/10-steps-to-pass-the-data-science-take-home-task-c5a696679526
- Notebook Template: https://medium.com/@jacowp357/12-steps-towards-implementing-a-data-science-solution-d2ce196f22c1

## Data Science Interview Questions
- Data Camp: https://www.datacamp.com/blog/data-scientist-interview-questions
- https://www.springboard.com/blog/data-science/data-science-interview-questions/

# Transformer models

## Before transformers: Static Embeddings

### One hot encoding

![one-hot](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*T5jaa2othYfXZX9W.)

### Word2Vec

![word2vec-vec](http://jalammar.github.io/images/word2vec/word2vec.png)

![word2vec](https://miro.medium.com/v2/resize:fit:1394/format:webp/0*XMW5mf81LSHodnTi.png)

### Проблема 1: статичних Embeddings недостатньо

- Приклад 1:
"Panda" + "Kung Fu" == "Kung Fu Panda" ?

Панда (в дикій природі), яка знає Кунг-фу != конкретний персонаж мультфільму "Панда Кунг-Фу"

- Приклад 2: The animal didn't cross the street because **it** was too tired.

Слово "воно" не існує незалежно від відповідного іменника.

### Проблема 2: RNNs працюють послідовно і тому повільно. Потрібна архітектура, яку легко паралелізувати
![rnn](https://miro.medium.com/v2/resize:fit:1400/format:webp/0*oY-GwnsZDEaHdVyf.gif)

## Transformer architecture(s)

Basic components:
- **attention** blocks
- fully connected layers

![components](http://jalammar.github.io/images/t/Transformer_encoder.png)

See a very good explanation here: http://jalammar.github.io/illustrated-transformer/

### Attention (self-attention)
![att](https://i.imgur.com/PHWQnbX.png)


### Attention: key + value + query

The key/value/query concept is analogous to retrieval systems. For example, when you search for videos on Youtube, the search engine will map your query (text in the search bar) against a set of keys (video title, description, etc.) associated with candidate videos in their database, then present you the best matched videos (values).

The attention operation can be thought of as a retrieval process as well.

Source: https://stats.stackexchange.com/questions/421935/what-exactly-are-keys-queries-and-values-in-attention-mechanisms

In [ ]:
# Attention

import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def attention(query, key, value):
    """Compute attention scores."""
    # Calculate dot product of query and key
    scores = np.dot(query, key)

    # Apply softmax to obtain attention weights
    attention_weights = softmax(scores)

    # Weighted sum of value vectors
    output = np.dot(attention_weights, value)

    return output, attention_weights

# Example input
query = np.array([0.5, 0.2, 0.1])  # Query vector
key = np.array([[0.8, 0.2, 0.1],    # Key vectors
                 [0.3, 0.6, 0.4],
                 [0.2, 0.5, 0.7]])
value = np.array([[3, 0],            # Value vectors
                  [5, 6],
                  [7, 8]])

# Compute attention
output, attention_weights = attention(query, key.T, value)

print("Attention output:", output)
print("Attention weights:", attention_weights)


In [ ]:
query

In [ ]:
key.T

In [ ]:
scores = np.dot(query, key.T)
scores

In [ ]:
attention_weights = softmax(scores)
attention_weights

In [ ]:
output = np.dot(attention_weights, value)
output

In [ ]:
# Calculate attention weights

from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Sentence
sentence = "The animal didn't cross the street because it was too tired."

# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print("Tokens:", tokens)

# Convert tokens to token IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print("Token IDs:", token_ids)

# Assuming query, key, and value are all token embeddings obtained from BERT
# For simplicity, let's assume query, key, and value are all randomly generated for each token
embeddings_dim = 128
query = np.random.rand(len(token_ids), embeddings_dim)
key = np.random.rand(len(token_ids), embeddings_dim)
value = np.random.rand(len(token_ids), embeddings_dim)

# Compute attention scores
outputs = []
attention_weights = []
for i in range(len(token_ids)):
    output, attn_weights = attention(query[i], key.T, value)
    outputs.append(output)
    attention_weights.append(attn_weights)

print("Attention output for each token:", outputs)
print("Attention weights for each token:", attention_weights)


In [ ]:
len(tokens) # tokens

In [ ]:
len(outputs[0])  # embeddings

In [ ]:
outputs[0]

In [ ]:
len(attention_weights[0]) # attention_weights of each word to each word

In [ ]:
attention_weights[0]

In [ ]:
import matplotlib.pyplot as plt

# Function to plot attention weights
def plot_attention_weights(attention_weights, tokens):
    fig, ax = plt.subplots()
    im = ax.imshow(attention_weights, cmap='viridis')

    # Set ticks for tokens
    ax.set_xticks(np.arange(len(tokens)))
    ax.set_yticks(np.arange(len(tokens)))
    # Label ticks with the respective list entries
    ax.set_xticklabels(tokens)
    ax.set_yticklabels(tokens)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            text = ax.text(j, i, round(attention_weights[i, j], 2), ha="center", va="center", color="black")

    ax.set_title("Attention Weights")
    fig.tight_layout()
    plt.show()

# Convert attention weights list to numpy array
attention_weights_array = np.array(attention_weights)

# Plot attention weights
plot_attention_weights(attention_weights_array, tokens)


In [ ]:
import tensorflow as tf
from keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Embedding
from keras.models import Model

class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(num_heads, d_model)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])

        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x, training=None):
        attn_output = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2

class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(TransformerEncoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = self.positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [TransformerEncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = Dropout(rate)

    def call(self, x, training=None):
        seq_len = tf.shape(x)[1]

        # Adding embedding and position encoding.
        x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training)

        return x

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'num_layers': self.num_layers,
            'd_model': self.d_model,
            'num_heads': self.num_heads,
            'dff': self.dff,
            'input_vocab_size': self.input_vocab_size,
            'maximum_position_encoding': self.maximum_position_encoding,
            'rate': self.rate,
        })
        return config

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)

        # apply sin to even indices in the array; 2i
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

        # apply cos to odd indices in the array; 2i+1
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]

        return tf.cast(pos_encoding, dtype=tf.float32)

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

# Example usage
num_layers = 6
d_model = 512
num_heads = 8
dff = 2048
input_vocab_size = 10000
maximum_position_encoding = 10000
rate = 0.1

inputs = Input(shape=(None,))
encoder = TransformerEncoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, maximum_position_encoding, rate)

outputs = encoder(inputs)
model = Model(inputs, outputs)

model.summary()


## Pretrained models and Finetuning

https://huggingface.co/learn/nlp-course/en/chapter1/4

![arch](https://textbook.edu.goit.global/python/data-science-remaster/v1/img/module-11/example5.png)

In [ ]:
# use DistilBERT out of the box
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset('imdb')

# Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
pretrained_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Example text for prediction
example_text = "This movie was great! I loved the acting and the storyline."

# Tokenize input text
inputs = tokenizer(example_text, return_tensors='pt')
inputs

In [ ]:
# Make prediction
outputs_pretrained = pretrained_model(**inputs)
predicted_label_pretrained = torch.argmax(outputs_pretrained.logits).item()

# Map predicted label to sentiment
sentiment_pretrained = 'positive' if predicted_label_pretrained == 1 else 'negative'

print("Predicted sentiment:", sentiment_pretrained)

In [ ]:
# Finetune DistilBERT

import os
os.environ["WANDB_DISABLED"] = "true"  # вимикаємо Weights & Biases

import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# 1. Завантаження IMDb датасету
dataset = load_dataset('imdb')

# 2. Перетворення в списки
texts = list(dataset['train']['text'])
labels = list(dataset['train']['label'])

# 3. Train/Test Split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

# 4. Завантаження токенайзера та моделі
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2
)

# 5. Токенізація
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

# 6. Torch Dataset
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

# 7. Метрика
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# 8. Параметри тренування
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs",
    logging_steps=100,
    save_strategy="epoch"
)

# 9. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# 10. Навчання
trainer.train()

# 11. Оцінка
print(trainer.evaluate())

# 12. Збереження моделі
model.save_pretrained("./distilbert-imdb")
tokenizer.save_pretrained("./distilbert-imdb")

# 13. Передбачення на своєму тексті
example_text = "This movie was absolutely fantastic! Great acting and story."
inputs = tokenizer(example_text, return_tensors="pt", truncation=True, padding=True)

model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=-1)

print("Probabilities:", probs)
print("Predicted class:", torch.argmax(probs).item())  # 1 = позитив, 0 = негатив


In [ ]:
model

In [ ]:
# Make prediction
# Example text for prediction
example_text = "This movie was great! I loved the acting and the storyline."

# Tokenize input text
inputs = tokenizer(example_text, return_tensors='pt')
# if you work on a mac:
#inputs = tokenizer(example_text, return_tensors='pt').to("mps")
outputs = model(**inputs)
predicted_label = torch.argmax(outputs.logits).item()

# Map predicted label to sentiment
sentiment = 'positive' if predicted_label == 1 else 'negative'

print("Predicted sentiment:", sentiment)

In [ ]:
from google.colab import files
import shutil

# Запакуємо папку з моделлю в zip
shutil.make_archive("distilbert-imdb", 'zip', "./distilbert-imdb")

# Завантажимо zip-файл на комп’ютер
files.download("distilbert-imdb.zip")


## From BERT to LLMs

https://arxiv.org/abs/2402.06196v2

# More resources

- Huggingface: https://huggingface.co (pretrained models, datasets, libraries)
- Tutorials from huggingface: https://huggingface.co/learn/nlp-course/chapter1/1
- Spacy: https://spacy.io/usage/embeddings-transformers#static-vectors
- For work with LLMs: Prompt engineering guide - https://www.promptingguide.ai